**Extracting Automotive Reviews from S3 data source**

In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,496 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-24 16:04:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-04-24 16:04:55 (9.38 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
automotive_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

automotive_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
#Show number of rows in the dataset
automotive_df.count()

3514942

**Transform**

In [6]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.
clean_df = automotive_df.dropna()
duplicate_df = clean_df.distinct()

In [7]:
#creating the review_id_table
review_id_table_df = duplicate_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100B3Y924RGIH|   14305020|B00B0GCZ3I|     881172530| 2014-05-29|
|R100BI6X9TR7G8|   11530488|B00CC2ZU4K|     954296244| 2015-01-21|
|R100IKQF3ORNJF|    9900991|B0091RCKPG|     240957722| 2014-01-10|
|R100PNUOGPNLHB|   22085873|B00EC4OUT2|     726003134| 2015-08-26|
|R100QSKTYYHYBD|   21945587|B001PF4ZZ0|     155818034| 2015-07-31|
|R100RIURK0DUFM|     659629|B000MSDKMA|     622787089| 2015-02-16|
|R100SPLJRY8XB2|    6733934|B005U30JXK|     937150161| 2015-08-11|
|R101691UN8LFDK|   21003951|B004DESMEK|      62726443| 2013-06-04|
|R1018Z926QMNB9|   41066228|B00EECM5FI|     667684414| 2015-01-26|
|R101OY4J4NHTGH|   52526844|B005VIIVZW|     211604382| 2014-04-17|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [8]:
# ensuring that dataFrames match in data type and in column name
review_id_table_df.schema

StructType(List(StructField(review_id,StringType,true),StructField(customer_id,IntegerType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(review_date,StringType,true)))

In [9]:
from pyspark.sql.types import * 

review_id_table_df = review_id_table_df.withColumn("review_date",review_id_table_df["review_date"].cast(DateType()))

review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
#Create products table
products_table_df = duplicate_df.select(["product_id", "product_title"]).distinct()

In [11]:
# ensuring that dataFrames match in data type and in column name
products_table_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
#create customers table and ensure that dataFrames match in data type and in column name
customers_table_df = duplicate_df.groupby("customer_id").count().withColumnRenamed('count', 'customer_count')

In [13]:
#ensuring that dataFrames match in data type and in column name
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
customers_table_df =  customers_table_df.select(["customer_id", F.col("customer_count").cast(IntegerType())])
customers_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [14]:
#create vine table and ensure that dataFrames match in data type and in column name
vine_table_df = duplicate_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



 **Load - DataFrames into an RDS instance**

In [15]:
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint REMOVED>:5432/bigdata_db"
config = {"user":"root",
          "password": "<PASSWORD REMOVED>",
          "driver":"org.postgresql.Driver"}

In [16]:
#Write dataframe to review_id_table
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
#Write dataframe to products_table_df
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [20]:
#Write dataframe to customers_table_df
customers_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:
#Write dataframe to vine_table_df
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)